# DATA MINING - WEEK 5
## NGUYEN XUAN VIET DUC
## 22280012 

### III. Practice content

#### 1. Using library

In [1]:
!pip install pyECLAT

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.0 MB/s eta 0:00:0031m893.6 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 21.0 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 21.0 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 20.2 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
from pyECLAT import ECLAT

data = pd.read_csv('data.csv', header=None)
data

,0,1,2,3,4,5
0,Wine,Chips,Bread,Butter,Milk,Apple
1,Wine,NaN,Bread,Butter,Milk,NaN
2,NaN,NaN,Bread,Butter,Milk,NaN
3,NaN,Chips,NaN,NaN,NaN,Apple
4,Wine,Chips,Bread,Butter,Milk,Apple
5,Wine,Chips,NaN,NaN,Milk,NaN
6,Wine,Chips,Bread,Butter,NaN,Apple
7,Wine,Chips,NaN,NaN,Milk,NaN
8,Wine,NaN,Bread,NaN,NaN,Apple
9,Wine,NaN,Bread,Butter,Milk,NaN


In [4]:
eclat_instance = ECLAT(data=data, verbose=True)
eclat_instance.df_bin

100%|████████████████████████████████████████████| 6/6 [00:00<00:00, 962.84it/s]


,Wine,Butter,Milk,Apple,Bread,Chips
0,1,1,1,1,1,1
1,1,1,1,0,1,0
2,0,1,1,0,1,0
3,0,0,0,1,0,1
4,1,1,1,1,1,1
5,1,0,1,0,0,1
6,1,1,0,1,1,1
7,1,0,1,0,0,1
8,1,0,0,1,1,0
9,1,1,1,0,1,0


In [7]:
items_per_transaction = eclat_instance.df_bin.astype(int).sum(axis=1)
min_support = 0.6
min_combination = 2
max_combination = max(items_per_transaction)
rule_indices, rule_supports = eclat_instance.fit(min_support=min_support, 
                                                 min_combination=min_combination, 
                                                 max_combination=max_combination, 
                                                 separator=' & ', verbose=True)
result = pd.DataFrame(rule_supports.items(), columns=['Item', 'Support'])
result1=result.sort_values(by=['Support'], ascending=False)
result1

Combination 2 by 2


15it [00:00, 246.35it/s]


Combination 3 by 3


20it [00:00, 404.93it/s]


Combination 4 by 4


...

Combination 5 by 5


6it [00:00, 436.69it/s]


Combination 6 by 6


1it [00:00, 391.19it/s]


,Item,Support
0,Wine & Milk,0.636364


#### 2. Vertical Apriori from scratch

In [8]:
from collections import defaultdict
from itertools import combinations
from typing import Dict, List, Set, Tuple, Any


class VerticalApriori:
    def __init__(self, min_support: float = 0.5):
        """
        Initialize the Vertical Apriori algorithm.
        
        Parameters:
        -----------
        min_support : float
            Minimum support threshold (between 0 and 1)
        """
        self.min_support = min_support
        self.transactions_count = 0
        self.freq_itemsets = []
        self.tid_mapping = {}  # Item to transaction ID mapping

    def _create_vertical_representation(self, dataset: List[List[Any]]) -> Dict[Any, Set[int]]:
        """
        Convert horizontal dataset to vertical representation.
        
        Parameters:
        -----------
        dataset : List[List[Any]]
            List of transactions where each transaction is a list of items
            
        Returns:
        --------
        Dict[Any, Set[int]]
            Dictionary mapping items to sets of transaction IDs
        """
        vertical_representation = defaultdict(set)
        
        for tid, transaction in enumerate(dataset):
            for item in transaction:
                vertical_representation[item].add(tid)
        
        return vertical_representation
    
    def _get_support(self, item_tids: Set[int]) -> float:
        """
        Calculate support for an itemset based on its transaction IDs.
        
        Parameters:
        -----------
        item_tids : Set[int]
            Set of transaction IDs containing the itemset
            
        Returns:
        --------
        float
            Support value
        """
        return len(item_tids) / self.transactions_count
    
    def _generate_candidate_itemsets(self, prev_freq_itemsets: List[Tuple], k: int) -> List[Tuple]:
        """
        Generate candidate k-itemsets from (k-1)-itemsets.
        
        Parameters:
        -----------
        prev_freq_itemsets : List[Tuple]
            List of frequent (k-1)-itemsets
        k : int
            Size of itemsets to generate
            
        Returns:
        --------
        List[Tuple]
            List of candidate k-itemsets
        """
        candidates = []
        
        # Self-joining step
        for i in range(len(prev_freq_itemsets)):
            for j in range(i + 1, len(prev_freq_itemsets)):
                # For k > 2, we join only if first k-2 elements are identical
                if k > 2:
                    if prev_freq_itemsets[i][:-1] == prev_freq_itemsets[j][:-1]:
                        # Create new candidate by combining the two itemsets
                        new_candidate = prev_freq_itemsets[i][:-1] + (prev_freq_itemsets[i][-1],) + (prev_freq_itemsets[j][-1],)
                        candidates.append(new_candidate)
                else:  # k == 2
                    new_candidate = (prev_freq_itemsets[i][0], prev_freq_itemsets[j][0])
                    candidates.append(new_candidate)
        
        # For k > 2, prune step: remove candidates with infrequent subsets
        if k > 2:
            pruned_candidates = []
            for candidate in candidates:
                should_prune = False
                # Check if all k-1 subsets are frequent
                for subset in combinations(candidate, k - 1):
                    if subset not in prev_freq_itemsets:
                        should_prune = True
                        break
                if not should_prune:
                    pruned_candidates.append(candidate)
            return pruned_candidates
        
        return candidates
    
    def _get_transaction_ids(self, itemset: Tuple) -> Set[int]:
        """
        Get transaction IDs containing an itemset by intersecting TIDs of individual items.
        
        Parameters:
        -----------
        itemset : Tuple
            The itemset to find transaction IDs for
            
        Returns:
        --------
        Set[int]
            Set of transaction IDs containing the itemset
        """
        if len(itemset) == 1:
            return self.tid_mapping[itemset[0]]
        
        # For multiple items, compute intersection of their TIDs
        result = self.tid_mapping[itemset[0]]
        for item in itemset[1:]:
            result = result.intersection(self.tid_mapping[item])
        
        return result
    
    def fit(self, dataset: List[List[Any]]) -> Dict[Tuple, float]:
        """
        Apply the vertical Apriori algorithm to find frequent itemsets.
        
        Parameters:
        -----------
        dataset : List[List[Any]]
            List of transactions where each transaction is a list of items
            
        Returns:
        --------
        Dict[Tuple, float]
            Dictionary mapping frequent itemsets to their support values
        """
        self.transactions_count = len(dataset)
        min_support_count = self.min_support * self.transactions_count
        
        # Create vertical representation
        self.tid_mapping = self._create_vertical_representation(dataset)
        
        # Find frequent 1-itemsets
        freq_1_itemsets = []
        support_data = {}
        
        for item, tids in self.tid_mapping.items():
            support = self._get_support(tids)
            if support >= self.min_support:
                freq_1_itemsets.append((item,))
                support_data[(item,)] = support
        
        self.freq_itemsets.append(freq_1_itemsets)
        
        k = 2
        while self.freq_itemsets[k-2]:
            # Generate candidate k-itemsets
            candidates = self._generate_candidate_itemsets(self.freq_itemsets[k-2], k)
            
            # Find frequent k-itemsets
            current_freq_itemsets = []
            
            for candidate in candidates:
                # Get transaction IDs containing the candidate
                candidate_tids = self._get_transaction_ids(candidate)
                support = self._get_support(candidate_tids)
                
                if support >= self.min_support:
                    current_freq_itemsets.append(candidate)
                    support_data[candidate] = support
            
            self.freq_itemsets.append(current_freq_itemsets)
            k += 1
        
        # Remove the last empty list
        self.freq_itemsets = self.freq_itemsets[:-1]
        
        return support_data
    
    def generate_rules(self, support_data: Dict[Tuple, float], min_confidence: float = 0.7) -> List[Tuple[Tuple, Tuple, float]]:
        """
        Generate association rules from frequent itemsets.
        
        Parameters:
        -----------
        support_data : Dict[Tuple, float]
            Dictionary mapping frequent itemsets to their support values
        min_confidence : float
            Minimum confidence threshold (between 0 and 1)
            
        Returns:
        --------
        List[Tuple[Tuple, Tuple, float]]
            List of rules as (antecedent, consequent, confidence)
        """
        rules = []
        
        for k in range(1, len(self.freq_itemsets)):
            # For each frequent itemset with size > 1
            for itemset in self.freq_itemsets[k]:
                if len(itemset) > 1:
                    # Generate all possible subsets for rules
                    for i in range(1, len(itemset)):
                        for antecedent in combinations(itemset, i):
                            # Convert antecedent to tuple if it's not already
                            antecedent_tuple = antecedent if isinstance(antecedent, tuple) else (antecedent,)
                            
                            # Calculate consequent by removing antecedent items from itemset
                            consequent = tuple(item for item in itemset if item not in antecedent)
                            
                            # Calculate confidence
                            confidence = support_data[itemset] / support_data[antecedent_tuple]
                            
                            if confidence >= min_confidence:
                                rules.append((antecedent_tuple, consequent, confidence))
        
        return rules

In [13]:
print("Original dataset:")
print(data.head())
print(f"Dataset shape: {data.shape}")

# Convert the dataset to transaction format (list of lists)
transactions = []
for _, row in data.iterrows():
    # Filter out NaN values and create a list of items
    transaction = [item for item in row.values if isinstance(item, str)]
    transactions.append(transaction)

print("\nTransactions format (first 5):")
for i, transaction in enumerate(transactions[:5]):
    print(f"Transaction {i}: {transaction}")

print(f"Total transactions: {len(transactions)}")

# Create and run the Vertical Apriori algorithm
min_support = 0.6  # 60% minimum support
print(f"\nRunning Vertical Apriori with minimum support = {min_support}")
apriori = VerticalApriori(min_support=min_support)
support_data = apriori.fit(transactions)

# Print frequent itemsets by size
print("\nFrequent Itemsets:")
for k, itemsets in enumerate(apriori.freq_itemsets):
    print(f"{k+1}-itemsets: {len(itemsets)} found")
    for itemset in sorted(itemsets):
        support = support_data[itemset]
        count = int(support * len(transactions))
        print(f"  {itemset}: support = {support:.2f} (found in {count} transactions)")

# Generate association rules with 80% minimum confidence
min_confidence = 0.8
print(f"\nGenerating association rules with minimum confidence = {min_confidence}")
rules = apriori.generate_rules(support_data, min_confidence=min_confidence)

# Print association rules
if rules:
    print(f"\nAssociation Rules ({len(rules)} rules found):")
    # Sort rules by confidence
    sorted_rules = sorted(rules, key=lambda x: x[2], reverse=True)
    for antecedent, consequent, confidence in sorted_rules:
        combined = antecedent + consequent
        support = support_data.get(combined, 0)
        consequent_str = ', '.join(consequent)
        antecedent_str = ', '.join(antecedent)
        print(f"  {antecedent_str} => {consequent_str}")
        print(f"    - confidence: {confidence:.2f}")
        print(f"    - support: {support:.2f}")
else:
    print("\nNo association rules found with the given confidence threshold.")

Original dataset:
      0      1       2       3     4      5
0  Wine  Chips  Bread   Butter  Milk  Apple
1  Wine    NaN  Bread   Butter  Milk    NaN
2   NaN    NaN  Bread   Butter  Milk    NaN
3   NaN  Chips     NaN     NaN   NaN  Apple
4  Wine  Chips  Bread   Butter  Milk  Apple
Dataset shape: (22, 6)

Transactions format (first 5):
Transaction 0: ['Wine', 'Chips', 'Bread ', 'Butter', 'Milk', 'Apple']
Transaction 1: ['Wine', 'Bread ', 'Butter', 'Milk']
Transaction 2: ['Bread ', 'Butter', 'Milk']
Transaction 3: ['Chips', 'Apple']
Transaction 4: ['Wine', 'Chips', 'Bread ', 'Butter', 'Milk', 'Apple']
Total transactions: 22

Running Vertical Apriori with minimum support = 0.6

Frequent Itemsets:
1-itemsets: 6 found
  ('Apple',): support = 0.68 (found in 14 transactions)
  ('Bread ',): support = 0.73 (found in 16 transactions)
  ('Butter',): support = 0.68 (found in 14 transactions)
  ('Chips',): support = 0.64 (found in 14 transactions)
  ('Milk',): support = 0.77 (found in 17 transactio